In [1]:
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql import Row
import pandas as pd

In [2]:
contest_data = sqlContext.sql("select * from edso_ignite.contest_data")

contest_cr = sqlContext.sql("select * from edso_ignite.contest_cr")

contest_savm = sqlContext.sql("select * from edso_ignite.contest_savm")

In [11]:
contest_savm_r = contest_savm.select(*(F.col(x).alias(x + '_savm') for x in contest_savm.columns))
join_a = contest_data.join(contest_savm_r,F.col('sales_acct_id')==F.col('sales_acct_id_savm')).select(['priorpartyname','party_name_savm']).distinct().toPandas()

In [23]:
unique_pairs = contest_data.join(contest_savm_r,F.col('sales_acct_id')==F.col('sales_acct_id_savm')).select(['priorpartyname','party_name_savm']).distinct()

In [25]:
def make_pairs(x):
    pairs = []
    words1 = treat_names(x[0])
    words2 = treat_names(x[1])
    for word1 in words1:
        for word2 in words2:
            if word1 != word2:
                pairs.append(((word1, word2), 1))
    return pairs
    

builder = unique_pairs.flatMap(make_pairs).reduceByKey(lambda x, y : x + y)

In [39]:
denominator = builder.map(lambda x : (x[0][0], x[1] ** 2)).reduceByKey(lambda x, y : x + y).map(lambda x : math.sqrt(x[1]))
builder.map(lambda x : (x[0], x[1])).join(denominator).take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 203 in stage 95.0 failed 4 times, most recent failure: Lost task 203.3 in stage 95.0 (TID 5218, hdprd-c01-r09-03.cisco.com): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_1480840206892_475990/container_e13_1480840206892_475990_01_000027/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_1480840206892_475990/container_e13_1480840206892_475990_01_000027/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_148084020689

In [37]:
import math
denominator = builder.map(lambda x : (x[0][0], x[1] ** 2)).reduceByKey(lambda x, y : x + y).map(lambda x : math.sqrt(x[1]))
normalized = builder.map(lambda x : (x[0], x[1])).join(denominator).mapValues(lambda x : x[0] / x[1]).take(10)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 231 in stage 83.0 failed 4 times, most recent failure: Lost task 231.3 in stage 83.0 (TID 4759, hdprd-c01-r03-14.cisco.com): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_1480840206892_475990/container_e13_1480840206892_475990_01_000051/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_1480840206892_475990/container_e13_1480840206892_475990_01_000051/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/mapr/tmp/hadoop-tmp/hadoop-mapr/nm-local-dir/usercache/hdpsndbx55/appcache/application_148084020689

In [12]:
print len(join_a)
join_a.head(50)

7418275


,priorpartyname,party_name_savm
0,BRIDGES SYSTEM INTEGRATION,FORT BELVOIR CAFE LLC
1,BRIDGES SYSTEM INTEGRATION,US ARMY NATIONAL GUARD
2,US ARMY FORCE MANAGEMENT SUPPORT AGENCY,THE ARMY US DEPT OF
3,TOUCHBASE UC LTD,THE AGRICULTURAL MORTGAGE CORPORATION PLC
4,TOUCHBASE UC LTD,SEABREEZE LEASING LTD
5,TOUCHBASE UC LTD,TPN GROUP LTD
6,TOUCHBASE UC LTD,IBOS FINANCE LTD
7,TOUCHBASE UC LTD,INSIGHT INVESTMENT
8,TOUCHBASE UC LTD,STT-BIRMINGHAM MIDSHIRES INDEPENDENT FINANCIAL...
9,TOUCHBASE UC LTD,AIRCLAIMS LTD


In [14]:
#name_freq_table = {} #comment out after first use, to prevent accidentally emptying it out

In [15]:
#creating function to create dictionary:
import string
stop_words = ['GROUP','LLC','INC','LTD','LIMITED','MEDIA','COUNTY','LLP','&','AND','AGENCY','LP','CORPORATION','COMPANY','DEPT','DEPTT','USA','CORP','COMMUNICATIONS','AMERICA','NORTH AMERICA', 'SYSTEMS', 
'FORUM', 'LOGISTICS', 'DEPT', 'SERVICES', 'OF', 'THE', 'NA', 'BRANDS', 'STATE','CORP','ST', 'SELECT', 'INTERNATIONAL','COMMERCIAL','HOLDINGS','HOLDING','ENTERPRISE']

def name_freq(row):
    global name_freq_table
    words1 = treat_names(row['priorpartyname'])
    words2 = treat_names(row['party_name_savm'])
    for word1 in words1:
        for word2 in words2:
            if word1 != word2:
                key = (word1,word2)
                if key in name_freq_table:
                    name_freq_table[key] += 1
                else:
                    name_freq_table[key] = 1
                    
def treat_names(name):
    name = ''.join(ch for ch in name if ch not in string.punctuation)
    resultwords  = [word for word in name.split() if word not in stop_words]
    return resultwords

In [16]:
join_a.apply(name_freq,axis=1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
7418245    None
7418246    None
7418247    None
7418248    None
7418249    None
7418250    None
7418251    None
7418252    None
7418253    None
7418254    None
7418255    None
7418256    None
7418257    None
7418258    None
7418259    None
7418260    None
7418261    None
7418262    None
7418263    None
7418264    None
7418265    None
7418266    None
7418267    None
7418268    None
7418269    None
7418270    None
7418271    None
7418272    None
7418273    None
7418274    None
dtype: object

In [19]:
sp_frequency_table = sqlContext.createDataFrame(dict(((k,name_freq_table[k]),1) for k in name_freq_table),['name_pair','count'])

IOError: [Errno 28] No space left on device

In [22]:
n = 10
{key:value for key,value in name_freq_table.items()[0:n]}

{(u'BORROWER', u'COMPRESSOR'): 1,
 (u'CISCO', u'HI'): 3,
 (u'CLAIMS', u'DCSIM'): 1,
 (u'CONFIG', u'CSD62454'): 1,
 (u'CUSTOMER', u'GETTY'): 21,
 (u'FO86', u'VNMB'): 1,
 (u'FRIBOURG', u'S'): 2,
 (u'INGHAM', u'ADVENTIST'): 2,
 (u'NETWORKS', u'COMMONWELATH'): 1,
 (u'SHAREHOLDER', u'PROCESSING'): 4}

In [18]:
sp_frequency_table.write.saveAsTable("ignite.contest_name_freq", mode = 'overwrite')

10705304

In [ ]:
pd_frequency_table = sp_frequency_table.toPandas()

In [ ]:
pd_frequency_table['word1'] = pd_frequency_table['name_pair'].map(lambda x: x[0])
pd_frequency_table['word2'] = pd_frequency_table['name_pair'].map(lambda x: x[1])

In [ ]:
pd_frequency_table.drop(['name_pair'],inplace=True)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(pd_frequency_table['word1'])
pd_frequency_table['word1'] =le.transform(pd_frequency_table['word1'])
pd_frequency_table['word2'] =le.transform(pd_frequency_table['word2'])

In [ ]:
from scipy.sparse import coo_matrix
freq_sparse_matrix = coo_matrix((pd_frequency_table['count'], (pd_frequency_table['word1'], pd_frequency_table['word2'])))